# COMP0119 - Assignment 2

### Importing libraries

In [1]:
# Loading libraries for visualisation
import matplotlib
import matplotlib.pyplot as plt

# Loading libraries for basic geometry processing
import trimesh
import numpy as np
from sklearn.neighbors import KDTree

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import open3d as o3d
import cv2
import scipy

from scipy.sparse import coo_matrix, csr_matrix

import numpy as np
import cv2
import open3d as o3d
from open3d import io
from open3d import utility

### Utilities

In [2]:
def trimesh_to_open3d(trimesh_mesh):
    """
    Convert a trimesh.Trimesh object to an open3d.geometry.TriangleMesh object.
    
    Args:
        trimesh_mesh: A trimesh.Trimesh object.
    
    Returns:
        An open3d.geometry.TriangleMesh object.
    """
    # Convert vertices and faces to Open3D format
    vertices = np.asarray(trimesh_mesh.vertices)
    faces = np.asarray(trimesh_mesh.faces)
    
    # Create an Open3D TriangleMesh object
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(faces)
    
    # Compute vertex normals if not already present
    if not o3d_mesh.has_vertex_normals():
        o3d_mesh.compute_vertex_normals()
    
    return o3d_mesh

In [3]:


def freedman_diaconis_bins(data, return_step=False):
    """
    Use the Freedman-Diaconis rule to compute the number of bins for a histogram.
    bin_width = 2 * IQR * (n ** (-1/3))
    where IQR is the interquartile range and n is the number of data points.
    
    Args:
        data:        1D array-like data to compute the bin width for.
        return_step: Whether to return the bin width (step) as well.
    
    Returns:
        Number of bins to use, and optionally the bin width (step).
    """
    n_data_points = len(data)
    IQR = np.subtract(*np.percentile(data, [75, 25]))
    bin_width = 2 * IQR * n_data_points ** (-1 / 3)
    num_bins = int((np.max(data) - np.min(data)) / bin_width)
    
    if return_step:
        return num_bins, bin_width
    else:
        return num_bins

def process_and_save_mesh(mesh, curvatures, file_name):
    # Convert trimesh object to Open3D mesh object
    o3d_mesh = trimesh_to_open3d(mesh)
    
    # Normalize curvature values to the range [0, 1]
    curv = curvatures.copy()
    curv -= np.min(curv)
    curv /= np.max(curv)
    
    # Calculate the number of bins using the Freedman-Diaconis rule
    bins = freedman_diaconis_bins(curv)
    
    # Histogram equalization
    hist, bin_edges = np.histogram(curv, bins=bins, range=[0, 1])
    cdf = hist.cumsum()
    cdf_normalized = cdf / cdf[-1]  # Normalize
    curv = np.interp(curv, bin_edges[:-1], cdf_normalized)
    
    # Compute HSV colors: Hue based on curvature, Saturation and Value set to 1
    hsv_colors = np.vstack((120 * (1 - curv), np.ones_like(curv) * 255, np.ones_like(curv) * 255)).T
    hsv_colors = hsv_colors.reshape((-1, 1, 3)).astype(np.uint8)
    
    # Convert HSV to RGB
    rgb_colors = cv2.cvtColor(hsv_colors, cv2.COLOR_HSV2RGB).reshape((-1, 3)) / 255.0
    
    # Update mesh vertex colors
    o3d_mesh.vertex_colors = utility.Vector3dVector(rgb_colors)
    
    # Save the mesh
    io.write_triangle_mesh(file_name, o3d_mesh)



## 1 Uniform Laplace

In [4]:

def uniform_laplace(mesh):
    mean_curvatures = np.zeros(mesh.vertices.shape[0])
    for i, vertex in enumerate(mesh.vertices):
        neighbors = mesh.vertex_neighbors[i]
        laplacian = np.mean(mesh.vertices[neighbors] - vertex, axis=0)
        mean_curvatures[i] = np.linalg.norm(laplacian) / 2
    return mean_curvatures

In [5]:
# lilium_s.obj
mesh_li = trimesh.load('meshes/curvatures/lilium_s.obj')
mean_curvatures = uniform_laplace(mesh_li)
print(mean_curvatures)
process_and_save_mesh(mesh_li, mean_curvatures, 'lilium_s_mean.obj')

[0.00373539 0.01178194 0.01064989 0.01845495 0.0058664  0.01048235
 0.01034497 0.00633187 0.01500296 0.01365398 0.02296671 0.01084309
 0.00542575 0.04479832 0.01610369 0.01155844 0.03207361 0.00555982
 0.00985751 0.01001868 0.00524109 0.00767216 0.01743738 0.00287332
 0.01347421 0.00516933 0.00559402 0.00556372 0.00905189 0.00554372
 0.00313131 0.00897144 0.01830754 0.00681048 0.00445076 0.0073366
 0.00438035 0.00350743 0.00788887 0.00550484 0.00502088 0.00768996
 0.0041684  0.01083186 0.00271733 0.01443431 0.00976345 0.0068563
 0.00697346 0.00819163 0.00805567 0.00441114 0.05368422 0.01600826
 0.03334901 0.01169736 0.0004445  0.03705315 0.02781126 0.01891422
 0.01936448 0.00468723 0.00313798 0.01005262 0.00673587 0.02146317
 0.02030111 0.00689246 0.0133429  0.00298606 0.01580965 0.01676183
 0.00365313 0.00770944 0.00463176 0.00801693 0.0136676  0.02963822
 0.00996991 0.00584839 0.0132582  0.00607998 0.0084662  0.01965474
 0.01531151 0.01417498 0.00605946 0.01418165 0.01365394 0.007016

In [6]:
# plane.obj
mesh_plane = trimesh.load('meshes/curvatures/plane.obj')
mean_curvatures = uniform_laplace(mesh_plane)
print(mean_curvatures)
process_and_save_mesh(mesh_plane, mean_curvatures, 'plane_mean.obj')

[1.57133269e-02 9.31696854e-03 9.31680084e-03 9.31685674e-03
 9.31696854e-03 9.31680084e-03 9.31685674e-03 9.31696854e-03
 9.31680084e-03 1.17849952e-02 9.31696854e-03 2.35702260e-07
 2.35702260e-07 1.66666667e-07 2.35702260e-07 2.35702260e-07
 1.66666667e-07 2.35702260e-07 2.35702260e-07 9.31680084e-03
 9.31680084e-03 2.35702260e-07 2.35702260e-07 1.66666667e-07
 2.35702260e-07 2.35702260e-07 1.66666667e-07 2.35702260e-07
 2.35702260e-07 9.31696854e-03 9.31685674e-03 1.66666667e-07
 1.66666667e-07 0.00000000e+00 1.66666667e-07 1.66666667e-07
 0.00000000e+00 1.66666667e-07 1.66666667e-07 9.31685674e-03
 9.31696854e-03 2.35702260e-07 2.35702260e-07 1.66666667e-07
 2.35702260e-07 2.35702260e-07 1.66666667e-07 2.35702260e-07
 2.35702260e-07 9.31680084e-03 9.31680084e-03 2.35702260e-07
 2.35702260e-07 1.66666667e-07 2.35702260e-07 2.35702260e-07
 1.66666667e-07 2.35702260e-07 2.35702260e-07 9.31696854e-03
 9.31685674e-03 1.66666667e-07 1.66666667e-07 0.00000000e+00
 1.66666667e-07 1.666666

### Gaussian curvature

In [7]:
def compute_vertex_area(mesh, vertex_index):
    # compute barycentric area (1/3 of adjacent faces area)
    area = 0
    for face_index in mesh.vertex_faces[vertex_index]:
        if face_index != -1:
            area += np.linalg.norm(np.cross(mesh.vertices[mesh.faces[face_index][1]] - mesh.vertices[mesh.faces[face_index][0]], mesh.vertices[mesh.faces[face_index][2]] - mesh.vertices[mesh.faces[face_index][0]])) / 6
    return area

In [8]:
def gaussian_curvature(mesh):
    gaussian_curvatures = np.zeros(mesh.vertices.shape[0])
    for i, vertex in enumerate(mesh.vertices):
        angle_deficit = 2 * np.pi
        faces = mesh.vertex_faces[i]
        for face in faces:
            if face == -1: continue  # Trimesh uses -1 for missing faces
            vertices = mesh.faces[face]
            # Compute angles for the current face
            for j in range(3):
                if vertices[j] == i:
                    u = mesh.vertices[vertices[(j + 1) % 3]] - vertex
                    v = mesh.vertices[vertices[(j + 2) % 3]] - vertex
                    angle = np.arccos(np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v)))
                    angle_deficit -= angle
                    break
        area = compute_vertex_area(mesh, i)  # Implement this based on the choice of area normalization
        gaussian_curvatures[i] = angle_deficit / area
    return gaussian_curvatures

In [9]:

gaussian_curvatures = gaussian_curvature(mesh_li)
print(gaussian_curvatures)
process_and_save_mesh(mesh_li, gaussian_curvatures, 'lilium_s_gaussian.obj')

[ 2.73555450e+00 -1.55038778e+00 -3.71682604e+00 -3.38712210e+00
 -6.04452331e+00 -2.10231556e+00 -9.07657217e-01  6.88637084e-01
  4.76818276e+00  3.55387668e-01  1.33683534e+03  8.39369225e+00
  6.77466631e+00  3.39479080e+02  1.00209577e+03  3.71940455e+00
  6.45025057e+02 -1.01652960e+00 -1.86004701e+00 -3.24457767e+00
 -1.88662003e+00 -4.93736393e+00 -8.61935834e-01 -9.57753914e-04
 -2.22233614e-01  3.81517621e+00 -1.01556077e+00 -4.80759834e+00
  1.99271280e-01 -1.84932458e+00 -2.58787622e+00  7.19288089e-01
  1.31326710e+00 -2.28108753e+00 -1.57527633e-01  1.55292095e+00
  1.81774536e+00 -5.72654451e-01 -3.00175617e+00  5.58740850e+00
 -2.33238763e+00  1.43016507e+00  2.89600057e+00  1.39255842e+00
 -1.03631735e+00  2.78273614e-01  4.01840793e+03  6.21107297e+00
  2.52767728e+00  1.06507661e+00  2.29759395e+00  2.57667138e+00
  2.58658677e+02 -1.07645791e+00  5.73848168e+02 -3.40075290e+00
 -2.19296415e+00  4.24621157e+02  5.85345058e+02  7.67288852e+02
 -8.57760892e-01 -1.73572

In [10]:
gaussian_curvatures = gaussian_curvature(mesh_plane)
print(gaussian_curvatures)
process_and_save_mesh(mesh_plane, gaussian_curvatures, 'plane_gaussian.obj')

[ 1.27237047e+04  5.65486678e+03  5.65492333e+03  5.65497988e+03
  5.65486678e+03  5.65492333e+03  5.65497988e+03  5.65486678e+03
  5.65492333e+03  2.54474094e+04  5.65486678e+03  0.00000000e+00
 -1.99838146e-13  0.00000000e+00  0.00000000e+00 -1.99838146e-13
  0.00000000e+00  0.00000000e+00 -1.99838146e-13  5.65492333e+03
  5.65492333e+03 -1.99838146e-13 -1.99838146e-13 -1.99841144e-13
 -1.99838146e-13 -1.99838146e-13 -1.99841144e-13 -1.99838146e-13
  9.99190730e-14  5.65486678e+03  5.65497988e+03  0.00000000e+00
 -1.99841144e-13  0.00000000e+00  0.00000000e+00 -1.99841144e-13
  0.00000000e+00  0.00000000e+00 -1.99841144e-13  5.65497988e+03
  5.65486678e+03  0.00000000e+00 -1.99838146e-13  0.00000000e+00
  0.00000000e+00 -1.99838146e-13  0.00000000e+00  0.00000000e+00
 -1.99838146e-13  5.65492333e+03  5.65492333e+03 -1.99838146e-13
 -1.99838146e-13 -1.99841144e-13 -1.99838146e-13 -1.99838146e-13
 -1.99841144e-13 -1.99838146e-13  9.99190730e-14  5.65486678e+03
  5.65497988e+03  0.00000

## 2 First and Second Fundamental forms

## 3 Non-uniform (Discrete Laplace-Beltrami)

In [11]:


def squared_edge_lengths(mesh):
    # Calculate the squared lengths of the edges for each face in the mesh.
    # Each face's edge lengths are stored in a matrix where rows correspond to faces.
    # The result is a matrix of size Mx3 where M is the number of faces.
    vecs = np.roll(mesh.vertices[mesh.faces], -1, axis=1) - mesh.vertices[mesh.faces]
    l2 = np.sum(vecs**2, axis=2)
    return l2

def doublearea(mesh):
    # Compute double the area of each face using the lengths squared from `squared_edge_lengths`.
    # The double area is preferred for numerical stability in computations that follow, such as the cotangent weights.
    l2 = squared_edge_lengths(mesh)
    s = 0.5 * (np.sqrt(l2[:, 0]) + np.sqrt(l2[:, 1]) + np.sqrt(l2[:, 2]))  # Semi-perimeter
    dblA = 2 * np.sqrt(s * (s - np.sqrt(l2[:, 0])) * (s - np.sqrt(l2[:, 1])) * (s - np.sqrt(l2[:, 2])))
    return dblA

def cotmatrix_entries(mesh):
    # Calculate cotangent weights for the cotangent Laplace matrix.
    # It involves computing the cotangent of angles opposite each vertex within each face, which are then used as weights.
    # The matrix is structured with one row per face and three cotangent weights per row.
    l2 = squared_edge_lengths(mesh)
    dblA = doublearea(mesh)
    i = (l2[:, 1] + l2[:, 2] - l2[:, 0]) / dblA
    j = (l2[:, 2] + l2[:, 0] - l2[:, 1]) / dblA
    k = (l2[:, 0] + l2[:, 1] - l2[:, 2]) / dblA
    return np.stack([i, j, k], axis=1) / 4.0  # Divide by 4 to account for each triangle contributing twice to the area.

def cotmatrix(mesh):
    # Assemble the cotangent Laplace matrix from cotangent weights computed per face.
    # The matrix is used in various geometry processing algorithms like smoothing or curvature estimation.
    # It is a sparse matrix where non-zero entries correspond to the cotangent weights of the mesh.
    m, n = mesh.faces.shape[0], mesh.vertices.shape[0]
    C = cotmatrix_entries(mesh)
    I, J, Vals = [], [], []
    for f, c in zip(mesh.faces, C):
        for i in range(3):
            I.extend([f[i], f[(i+1)%3], f[i], f[(i+1)%3]])
            J.extend([f[(i+1)%3], f[i], f[i], f[(i+1)%3]])
            Vals.extend([c[i], c[i], -c[i], -c[i]])  # Negative weights for diagonal entries.
    L = coo_matrix((Vals, (I, J)), shape=(n, n))
    return csr_matrix(L)  # Convert to CSR format for efficient arithmetic and matrix-vector operations.

def compute_vertex_areas(mesh):
    # Compute the area associated with each vertex by summing the areas of adjacent faces and dividing by 3 (since each face is a triangle).
    # This is used to weight the contributions of faces to the vertex in the computation of curvature and other measures.
    vertex_areas = np.zeros(len(mesh.vertices))
    for face in mesh.faces:
        edge_vectors = np.diff(mesh.vertices[face[np.array([0, 1, 2, 0])]], axis=0)
        face_area = np.linalg.norm(np.cross(edge_vectors[0], edge_vectors[1])) / 2
        vertex_areas[face] += face_area / 3  # Area of the face is evenly distributed to the vertices.
    return vertex_areas





In [12]:

def mean_curvature(mesh):
    L = cotmatrix(mesh)
    M_inv = scipy.sparse.diags(1.0 / compute_vertex_areas(mesh))
    HN = -M_inv.dot(L.dot(mesh.vertices))
    H = 0.5 * np.linalg.norm(HN, axis=1)
    return H



In [13]:

cotH_li = mean_curvature(mesh_li)
process_and_save_mesh(mesh_li, cotH_li, 'cot_mean_li.obj')

[Open3D WARNING] Write OBJ can not include triangle normals.


In [14]:

cotH_plane = mean_curvature(mesh_plane)
process_and_save_mesh(mesh_plane, cotH_plane, 'cot_mean_plane.obj')

[Open3D WARNING] Write OBJ can not include triangle normals.


## 4 Modal analysis

In [15]:

from scipy.sparse.linalg import eigs

def reconstruct_spectral_mesh(V, eigvecs):
    V_spectral = np.dot(eigvecs.T,V)
    # Reconstruct the vertex positions from the spectral domain
    V_reconstructed = np.dot(eigvecs,V_spectral)
    return V_reconstructed



def reconstruct_mesh1(mesh,k):
    L = cotmatrix(mesh)  # Compute the Laplace-Beltrami operator
    _, eigvecs = eigs(L, k=k, which='SM')
    eigvecs = eigvecs.real
    V_reconstructed = reconstruct_spectral_mesh(mesh.vertices, eigvecs)
    mesh.vertices = V_reconstructed.real

    return mesh


In [ ]:
mesh = trimesh.load('meshes/decomposition/armadillo.obj')
recon_mesh = reconstruct_mesh1(mesh,100)
recon_mesh.export('recon_100.obj')


## 5 explicit Laplacian mesh smoothing 

$$X^{n+1} = X^{n} + \lambda LX^{n}$$
where, $L = \frac{1}{|\mathcal{N}_1(v_i)|}\sum\limits_{v_j\in\mathcal{N}_1(v_i)}(f(v_j)-f(v_i))$ is the uniform lapalace beltrami operator (as the lectrue note #6), $X^i$ is the coordinate of the $i$-th point.

In [ ]:
def compute_uniform_laplace_beltrami_operator(mesh):

    n_vertices = len(mesh.vertices)
    
    # Indices for rows and columns of the matrix
    row_indices = []
    col_indices = []
    data = []
    
    for i in range(n_vertices):
        # Get neighbors for the ith vertex
        neighbors = mesh.vertex_neighbors[i]
        neighbor_count = len(neighbors)
        if neighbor_count > 0:
            uniform_weight = 1.0 / neighbor_count
            # For each neighbor, add the (i, j) coordinate to the sparse matrix, with uniform weight
            for j in neighbors:
                row_indices.append(i)
                col_indices.append(j)
                data.append(uniform_weight)  # Using uniform weight here
            # Add the (i, i) coordinate with a weight of -1
            row_indices.append(i)
            col_indices.append(i)
            data.append(-1)
    
    # Create the sparse matrix
    laplacian_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(n_vertices, n_vertices))
    
    return laplacian_matrix


In [ ]:
# implementation of explicit Laplacian mesh smoothing
def explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step=0.1, iterations=10):

    # get the original vertices coordinates
    smooth_vertices = mesh.vertices.copy()

    # Iterate over each vertex
    for i in range(iterations):
        smooth_vertices += lambda_step * laplace_matrix.dot(smooth_vertices)
    return smooth_vertices

In [ ]:
# load mesh
mesh_path = 'meshes/smoothing/plane_ns.obj'
mesh = trimesh.load(mesh_path, process=False)

In [ ]:
lambda_step = 0.1  # step size
iterations = 50  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("plane_ns_0.1_50.obj")

In [ ]:
lambda_step = 0.05  # step size
iterations = 50  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("plane_ns_0.05_50.obj")

In [ ]:
lambda_step = 0.01  # step size
iterations = 50  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("plane_ns_0.01_50.obj")

In [ ]:
lambda_step = 1  # step size
iterations = 50  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("plane_ns_1_50.obj")

In [ ]:
lambda_step = 2  # step size
iterations = 50  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("plane_ns_2_50.obj")

In [ ]:
# load mesh
mesh_path = 'meshes/smoothing/fandisk_ns.obj'
mesh = trimesh.load(mesh_path, process=False)

In [ ]:
lambda_step = 0.01  # step size
iterations = 50  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_0.01_50.obj")

In [ ]:
lambda_step = 0.05  # step size
iterations = 50  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_0.05_50.obj")

In [ ]:
lambda_step = 2  # step size
iterations = 50  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_2_50.obj")

In [ ]:
lambda_step = 0.05  # step size
iterations = 10  # number of iterations
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = explicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_0.05_10.obj")

## 6 implicit Laplacian mesh smoothing

$$(I - \lambda L)X^{(n+1)} = X^n $$

as the eqution (9) in the *Implicit Fairing of Irregular Meshes using Diffusion and Curvature Flow* , which integrating the diffusion equation with a simple explicit Euler scheme.

In [ ]:
# implementation of implicit Laplacian mesh smoothing
def implicit_Laplacian_mesh_smoothing(mesh, lpalace_matrix, lambda_step=0.1, iterations=10):

    smooth_vertices = mesh.vertices.copy()
    identity_matrix = eye(smooth_vertices.shape[0])


    A = identity_matrix - lambda_step * lpalace_matrix
    b = smooth_vertices
    
    # iterate over each vertex
    for i in range(iterations):
        # sparse linear solver to (I - dt * L) X^{n+1} = X^n
        smooth_vertices = scipy.sparse.linalg.spsolve(A, b)
    
    return smooth_vertices

In [ ]:
# load mesh
mesh_path = 'meshes/smoothing/fandisk_ns.obj'
mesh = trimesh.load(mesh_path, process=False)

In [ ]:
lambda_step = 0.01 
iterations = 50  
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = implicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_0.01_50_i.obj")

In [ ]:
lambda_step = 0.05
iterations = 50  
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = implicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_0.05_50_i.obj")

In [ ]:
lambda_step = 0.1
iterations = 50  
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = implicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_0.1_50_i.obj")

In [ ]:
lambda_step = 2
iterations = 50  
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = implicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_2_50_i.obj")

In [ ]:
lambda_step = 10
iterations = 50  
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = implicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_10_50_i.obj")

In [ ]:
lambda_step = 100
iterations = 50  
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = implicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_100_50_i.obj")

In [ ]:
lambda_step = 5
iterations = 50  
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = implicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_5_50_i.obj")

In [ ]:
lambda_step = 5
iterations = 10  
laplace_matrix = compute_uniform_laplace_beltrami_operator(mesh)
smoothed_vertices = implicit_Laplacian_mesh_smoothing(mesh, laplace_matrix, lambda_step, iterations)

smoothed_mesh = mesh.copy()
smoothed_mesh.vertices = smoothed_vertices
smoothed_mesh.export("fandisk_ns_5_10_i.obj")

## 7 Evaluate the performance 

In [ ]:
def add_noise_to_vertices(vertices, level):
    noisy_vertices = vertices + np.random.normal(0, level, vertices.shape)
    return noisy_vertices

In [ ]:
mesh = trimesh.load('meshes/smoothing/fandisk_ns.obj')
noisy_mesh = mesh.copy()
level = 0.05
noisy_vertices = add_noise_to_vertices(noisy_mesh.vertices, level)
noisy_mesh.vertices = noisy_vertices
noisy_mesh.export('noiys_0.05.obj')

In [ ]:

laplace_matrix = compute_uniform_laplace_beltrami_operator(noisy_mesh)

ex_smooth_mesh_vertices = explicit_Laplacian_mesh_smoothing(noisy_mesh, laplace_matrix, 0.05, 50)
ex_smoothed_mesh = noisy_mesh.copy()
ex_smoothed_mesh.vertices = ex_smooth_mesh_vertices
ex_smoothed_mesh.export('noiys_0.05_ex.obj')
im_smooth_mesh_vertices = implicit_Laplacian_mesh_smoothing(noisy_mesh, laplace_matrix, 5, 50)
im_smoothed_mesh = noisy_mesh.copy()
im_smoothed_mesh.vertices = im_smooth_mesh_vertices
im_smoothed_mesh.export('noiys_0.05_im.obj')


In [ ]:
noisy_mesh = mesh.copy()
level = 0.1
noisy_vertices = add_noise_to_vertices(noisy_mesh.vertices, level)
noisy_mesh.vertices = noisy_vertices
noisy_mesh.export('noiys_0.1.obj')

In [ ]:
laplace_matrix = compute_uniform_laplace_beltrami_operator(noisy_mesh)

ex_smooth_mesh_vertices = explicit_Laplacian_mesh_smoothing(noisy_mesh, laplace_matrix, 0.05, 50)
ex_smoothed_mesh = noisy_mesh.copy()
ex_smoothed_mesh.vertices = ex_smooth_mesh_vertices
ex_smoothed_mesh.export('noiys_0.1_ex.obj')
im_smooth_mesh_vertices = implicit_Laplacian_mesh_smoothing(noisy_mesh, laplace_matrix, 5, 50)
im_smoothed_mesh = noisy_mesh.copy()
im_smoothed_mesh.vertices = im_smooth_mesh_vertices
im_smoothed_mesh.export('noiys_0.1_im.obj')

In [ ]:
noisy_mesh = mesh.copy()
level = 0.5
noisy_vertices = add_noise_to_vertices(noisy_mesh.vertices, level)
noisy_mesh.vertices = noisy_vertices
noisy_mesh.export('noiys_0.5.obj')

In [ ]:
laplace_matrix = compute_uniform_laplace_beltrami_operator(noisy_mesh)

ex_smooth_mesh_vertices = explicit_Laplacian_mesh_smoothing(noisy_mesh, laplace_matrix, 0.05, 50)
ex_smoothed_mesh = noisy_mesh.copy()
ex_smoothed_mesh.vertices = ex_smooth_mesh_vertices
ex_smoothed_mesh.export('noiys_0.5_ex.obj')
im_smooth_mesh_vertices = implicit_Laplacian_mesh_smoothing(noisy_mesh, laplace_matrix, 5, 50)
im_smoothed_mesh = noisy_mesh.copy()
im_smoothed_mesh.vertices = im_smooth_mesh_vertices
im_smoothed_mesh.export('noiys_0.5_im.obj')